# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

### Also - adding DeepSeek if you wish

Optionally, if you'd like to also use DeepSeek, create an account [here](https://platform.deepseek.com/), create a key [here](https://platform.deepseek.com/api_keys) and top up with at least the minimum $2 [here](https://platform.deepseek.com/top_up).

### Adding API keys to your .env file

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
DEEPSEEK_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyB5


In [5]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [6]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [19]:
system_message = "You are a helpful assistant that is great at solving puzzles"
user_prompt = "On a bookshelf, two volumes of Pushkin stand side by side: the first and the second. The pages of each volume together have a thickness of 2 cm, and each cover is 2 mm thick. A worm gnawed (perpendicular to the pages) from the first page of the first volume to the last page of the second volume. What distance did it gnaw through?"

In [20]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [22]:
# GPT-4o-mini

completion = openai.chat.completions.create(model='gpt-4o-mini', messages=prompts)
print(completion.choices[0].message.content)

Let's calculate the thickness of the two volumes and the worm's path from the first page of the first volume to the last page of the second volume.

1. **Thickness of each volume**:
   - The pages of each volume have a total thickness of 2 cm, which is equal to 20 mm.

2. **Thickness of the covers for each volume**:
   - Each cover is 2 mm thick. 
   - Therefore, each volume (with covers) has a total thickness of:
     - Thickness of pages + Thickness of front cover + Thickness of back cover
     - \( 20 \text{ mm} + 2 \text{ mm} + 2 \text{ mm} = 24 \text{ mm} \)

3. **Total thickness for both volumes**:
   - Since there are two volumes, the total thickness of the two volumes together is:
     - \( 24 \text{ mm} + 24 \text{ mm} = 48 \text{ mm} \)

4. **Worm's path**:
   - The worm starts at the first page of the first volume. This means it begins at the very front of the first volume (after the front cover).
   - The worm then gnaws through to the last page of the second volume. The la

In [21]:
# GPT-4.1-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Let's analyze the problem step-by-step.

---

### Given:

- There are two volumes of Pushkin standing **side by side**: Volume 1 (the first) and Volume 2 (the second).
- The **total thickness of pages** in each volume is 2 cm.
- Each **cover** (front or back) is 2 mm thick.
- A worm gnawed **perpendicular to the pages** from the first page of the first volume to the last page of the second volume.
- We need to find the distance the worm traveled **inside the books** (through the covers and pages).

---

### Step 1: Understand the book structure

Each volume contains:

- Front cover: 2 mm thick = 0.2 cm
- Pages: 2 cm thick
- Back cover: 2 mm thick = 0.2 cm

Hence, thickness of each volume:

\[
\text{Volume thickness} = 0.2 + 2 + 0.2 = 2.4 \text{ cm}
\]

---

### Step 2: How are the books arranged on the shelf?

Books are placed **side by side**. 

Usually, the book front cover faces the left side, pages inside, back cover on the right side.

So:

- Volume 1 from left to right: front cov

In [23]:
# GPT-4.1-nano - extremely fast and cheap

completion = openai.chat.completions.create(
    model='gpt-4.1-nano',
    messages=prompts
)
print(completion.choices[0].message.content)

Let's carefully analyze the problem:

**Given data:**
- There are two volumes of Pushkin's works, placed side by side.
- The total thickness of the pages in each volume: **2 cm**.
- Thickness of each cover: **2 mm**.
- The worm gnaws from the **first page of the first volume** to the **last page of the second volume**.
- The worm gnaws **perpendicular to the pages** (i.e., straight through the covers and pages along the thickness direction).

---

### Step 1: Determine the thickness of each volume

Each volume consists of:
- Two covers (front and back): \(2 \text{ mm} + 2 \text{ mm} = 4 \text{ mm}\).
- Pages: 2 cm = 20 mm.

**Total thickness of each volume:**

\[
\text{Total per volume} = \text{covers} + \text{pages} + \text{covers} = 4 \text{ mm} + 20 \text{ mm} + 4 \text{ mm} = 28 \text{ mm}
\]

But since the pages are in the middle, the total thickness is:

\[
\boxed{28 \text{ mm}}
\]

---

### Step 2: Visualize the setup

The two volumes are placed side-by-side, aligned at the cove

In [24]:
# GPT-4.1

completion = openai.chat.completions.create(
    model='gpt-4.1',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Let's clarify the situation and solve the puzzle step by step.

**Books on a shelf:**
- Two volumes: Volume 1 and Volume 2, standing side by side in the usual way (left to right: Volume 1, then Volume 2).
- Each **set of pages** is **2 cm thick**.
- Each **cover** is **2 mm thick**. (There are two covers per volume: front and back.)

**The Worm's Path:**
- It starts **at the first page of Volume 1** and ends **at the last page of Volume 2**.
- The worm gnaws **perpendicular to the pages**.

**Key Point (Trick):**
The way books are arranged means that **the first page of Volume 1** and **the last page of Volume 2** are **closest together**, not farthest apart.

Let’s see why:

### How books are arranged
On a bookshelf, the **front cover of Volume 1** is at the far **left**. The **pages of Volume 1** come next. The **back cover of Volume 1** is in the middle, then the **front cover of Volume 2**, then the **pages of Volume 2**, then the **back cover of Volume 2** at the far **right**.

B

In [25]:
# If you have access to this, here is the reasoning model o3-mini
# This is trained to think through its response before replying
# So it will take longer but the answer should be more reasoned - not that this helps..

completion = openai.chat.completions.create(
    model='o3-mini',
    messages=prompts
)
print(completion.choices[0].message.content)

We must picture two volumes standing side by side. Each volume consists of a pages block, 2 cm (20 mm) thick, plus a front cover and a back cover that are each 2 mm thick. Thus, if you took an entire volume it would be 20 mm + 2 mm + 2 mm = 24 mm thick.

However, the worm’s journey did not start at an outer edge. It began at the very first page of the first volume—that is, just inside its front cover—and ended at the very last page of the second volume—that is, just before its back cover. In other words, the worm did not chew through the first volume’s front cover or the second volume’s back cover.

Let’s picture this step‐by‐step:

1. In the first volume the worm starts at the first page (inside the front cover). It must pass through the entire 2 cm (20 mm) of pages and then through the back cover (2 mm).

2. In the second volume the worm first goes through its front cover (2 mm) and then through its 2 cm (20 mm) of pages, stopping at the last page (so it does not go through the back 

In [27]:
# Claude 3.7 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=1000,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

I'll solve this step by step.

First, let me visualize the bookshelf with the two volumes of Pushkin standing side by side in order:
- Volume 1 (on the left)
- Volume 2 (on the right)

For each volume:
- Pages have a total thickness of 2 cm = 20 mm
- Each cover is 2 mm thick
- Each book has a front cover and a back cover

Now, the worm starts at the first page of Volume 1 and gnaws through to the last page of Volume 2.

Let's track what the worm encounters:
1. Starting at the first page of Volume 1, which is just after the front cover
2. The worm gnaws through all pages of Volume 1 (20 mm)
3. Then through the back cover of Volume 1 (2 mm)
4. Then through the front cover of Volume 2 (2 mm)
5. Finally through all pages of Volume 2 (20 mm), ending at the last page

So the total distance gnawed is:
20 mm + 2 mm + 2 mm + 20 mm = 44 mm = 4.4 cm

However, I need to be more careful about what "first page" and "last page" mean in this context. Since the books are arranged in order on the shelf,

In [28]:
# Claude 3.7 Sonnet again
# Now let's add in streaming back results
# If the streaming looks strange, then please see the note below this cell!

result = claude.messages.stream(
    model="claude-3-7-sonnet-latest",
    max_tokens=1000,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

I need to figure out the distance the worm traveled from the first page of volume 1 to the last page of volume 2.

Let me analyze how the books are positioned on the shelf:
- The books are side by side, with volume 1 on the left and volume 2 on the right
- Each volume has pages with a total thickness of 2 cm (20 mm)
- Each cover is 2 mm thick
- Each book has a front cover and a back cover

For volume 1:
- The first page is right after the front cover
- The last page is right before the back cover

For volume 2:
- The first page is right after the front cover
- The last page is right before the back cover

Now, let's trace the worm's path from the first page of volume 1 to the last page of volume 2:
1. The worm starts at the first page of volume 1, which is just after the front cover
2. It needs to travel through all pages of volume 1 (2 cm = 20 mm)
3. Then through the back cover of volume 1 (2 mm)
4. Then through the front cover of volume 2 (2 mm)
5. Finally through all pages of volume

## A rare problem with Claude streaming on some Windows boxes

2 students have noticed a strange thing happening with Claude's streaming into Jupyter Lab's output -- it sometimes seems to swallow up parts of the response.

To fix this, replace the code:

`print(text, end="", flush=True)`

with this:

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

And it should work fine!

In [29]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Here's how to solve this:

*   **Volume thickness:** Each volume's pages are 2 cm thick, which is 20 mm.
*   **Cover thickness:** Each cover is 2 mm thick.
*   **Worm's path:** The worm goes through the back cover of the first volume, all the pages of the first volume, the pages of the second volume, and the front cover of the second volume.

Therefore, the total distance the worm gnawed is: 2 mm + 20 mm + 20 mm + 2 mm = 44 mm.

**Answer:** The worm gnawed through 44 mm.


In [30]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google released endpoints that means you can use Gemini via the client libraries for OpenAI!
# We're also trying Gemini's latest reasoning/thinking model

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.5-flash-preview-04-17",
    messages=prompts
)
print(response.choices[0].message.content)

This is a classic riddle! The trick lies in understanding how the books are arranged on the shelf and the worm's start and end points.

1.  **Visualize the Books:** The two volumes stand side by side on a bookshelf. Since it's Volume 1 and Volume 2, Volume 1 is usually on the left and Volume 2 is on the right. The spines are facing outwards.
2.  **Visualize the Worm's Path:** The worm starts on the *first page* of Volume 1 and gnaws to the *last page* of Volume 2. "Perpendicular to the pages" means it's gnawing through the thickness of the book blocks, horizontally across the shelf from Volume 1 to Volume 2.
3.  **Identify Start and End Points:**
    *   Starting on the "first page" means the worm is *just inside* the front cover of Volume 1. It does *not* need to gnaw through the front cover of Volume 1.
    *   Ending on the "last page" means the worm is *just inside* the back cover of Volume 2. It does *not* need to gnaw through the back cover of Volume 2.
4.  **Determine What the W

## (Optional) Trying out the DeepSeek model

### Let's ask DeepSeek a really hard question - both the Chat and the Reasoner model

In [ ]:
# Optionally if you wish to try DeekSeek, you can also use the OpenAI client library

deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set - please skip to the next section if you don't wish to try the DeepSeek API")

In [ ]:
# Using DeepSeek Chat

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts,
)

print(response.choices[0].message.content)

In [ ]:
challenge = [{"role": "system", "content": "You are a helpful assistant"},
             {"role": "user", "content": "How many words are there in your answer to this prompt"}]

In [ ]:
# Using DeepSeek Chat with a harder question! And streaming results

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

print("Number of words:", len(reply.split(" ")))

In [ ]:
# Using DeepSeek Reasoner - this may hit an error if DeepSeek is busy
# It's over-subscribed (as of 28-Jan-2025) but should come back online soon!
# If this fails, come back to this in a few days..

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)
print("Number of words:", len(content.split(" ")))

## Additional exercise to build your experience with the models

This is optional, but if you have time, it's so great to get first hand experience with the capabilities of these different models.

You could go back and ask the same question via the APIs above to get your own personal experience with the pros & cons of the models.

Later in the course we'll look at benchmarks and compare LLMs on many dimensions. But nothing beats personal experience!

Here are some questions to try:
1. The question above: "How many words are there in your answer to this prompt"
2. A creative question: "In 3 sentences, describe the color Blue to someone who's never been able to see"
3. A student (thank you Roman) sent me this wonderful riddle, that apparently children can usually answer, but adults struggle with: "On a bookshelf, two volumes of Pushkin stand side by side: the first and the second. The pages of each volume together have a thickness of 2 cm, and each cover is 2 mm thick. A worm gnawed (perpendicular to the pages) from the first page of the first volume to the last page of the second volume. What distance did it gnaw through?".

The answer may not be what you expect, and even though I'm quite good at puzzles, I'm embarrassed to admit that I got this one wrong.

### What to look out for as you experiment with models

1. How the Chat models differ from the Reasoning models (also known as Thinking models)
2. The ability to solve problems and the ability to be creative
3. Speed of generation


## Back to OpenAI with a serious question

In [ ]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [4]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

NameError: name 'openai' is not defined

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [ ]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [ ]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>